<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/GAN_GPT%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential

# 하이퍼파라미터 설정
latent_dim = 100
image_shape = (28, 28, 1)

# 생성자 모델 정의
def build_generator(latent_dim):
    model = Sequential([
        Dense(256, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(512),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(1024),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(np.prod(image_shape), activation='tanh'),
        Reshape(image_shape)
    ])
    return model

# 판별자 모델 정의
def build_discriminator(image_shape):
    model = Sequential([
        Flatten(input_shape=image_shape),
        Dense(512),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

# 모델 생성
generator = build_generator(latent_dim)
discriminator = build_discriminator(image_shape)

# 판별자 컴파일
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 생성자-판별자 모델 연결 (GAN 모델)
discriminator.trainable = False
gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# 학습 루프
import numpy as np

def train_gan(epochs, batch_size, latent_dim):
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # 진짜 이미지로 훈련
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[idx]

        # 가짜 이미지로 훈련
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_images, valid)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # 생성자 훈련
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, valid)

        print(f"{epoch + 1}/{epochs} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

# 모델 학습
train_gan(epochs=10000, batch_size=64, latent_dim=latent_dim)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step 


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1/10000 [D loss: 0.6196648478507996, acc.: 65.625%] [G loss: [array(0.64860773, dtype=float32), array(0.64860773, dtype=float32), array(0.5625, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


2/10000 [D loss: 0.6322287917137146, acc.: 61.328125%] [G loss: [array(0.6447081, dtype=float32), array(0.6447081, dtype=float32), array(0.5703125, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
3/10000 [D loss: 0.6525346040725708, acc.: 55.546873807907104%] [G loss: [array(0.67249423, dtype=float32), array(0.67249423, dtype=float32), array(0.5078125, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
4/10000 [D loss: 0.6827675104141235, acc.: 49.8046875%] [G loss: [array(0.70380193, dtype=float32), array(0.70380193, dtype=float32), array(0.46484375, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
5/10000 [D loss: 0.7166999578475952, acc.: 46.51041626930237%] [G loss: [array(0.74117965, dtype=float32), array(0.74117965, dtype=float32), array(0.440625, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/10000 [D loss: 0.7567266225814819, acc.: 44.921875%] [G loss: [array(0.78742665, dtype=float32), array(0.78742665, dtype=float32), array(0.4296875, dtype=float32)